In [1]:
pip install selenium


     ------------------------------------ 981.4/981.4 kB 941.0 kB/s eta 0:00:00
     -------------------------------------- 359.0/359.0 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 55.8/55.8 kB 2.8 MB/s eta 0:00:00


In [2]:
pip install xlrd

     -------------------------------------- 96.5/96.5 kB 690.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver-manager

In [5]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import json
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, SoupStrainer
import re

In [6]:
sheet_id='1BZSPhk1LDrx8ytywMHWVpCqbm8URTxTJrIRkD7PnGTM'
df=pd.read_csv(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv")
df    

,Unnamed: 0,id,Asin,country
0,0,1,1015,de
1,1,2424796,1015,fr
2,2,2,000004458X,de
3,3,2424797,000004458X,fr
4,4,3,1002198,de
...,...,...,...,...
995,995,2425286,4484924,fr
996,996,449,4485742,de
997,997,2425287,4485742,fr
998,998,450,4486072,de


In [7]:
#https://www.amazon.{country}/dp/{asin}
asin=df['Asin']
code=df['country']
column=['product_name','product_price','product_detail','product_url']
Product=pd.DataFrame(columns=column)

In [9]:
for i in range(101):
    url="https://www.amazon.{}/dp/{}".format(code[i],asin[i])
    try:
        
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,"html.parser")
        #print(soup.prettify())
        #dict_from_json = json.loads(soup.find("body").text)
        name= driver.find_element(By.XPATH,"//span[contains(@id,'productTitle')]")
        price= driver.find_element(By.XPATH,"//span[contains(@class,'a-size-base a-color-price a-color-price')] | //span[contains(@class,'a-price-whole')]" )
        img_url=WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//span[@class='a-list-item']/span[@class='a-declarative']/div[@class='imgTagWrapper']/img[@class='a-dynamic-image a-stretch-horizontal'] | //div[@class='a-row a-spacing-mini a-spacing-top-micro']/span[@class='a-declarative']/div[contains(@class,'a-column a-span3 a-spacing-micro imageThumb thumb')]/img"))).get_attribute("src")
        detail= driver.find_element(By.XPATH,"//span[contains(@class,'a-list-item')] | //div[contains(@class,'a-section a-spacing-small a-padding-base')] ")
        Product=Product.append({'product_name':name.text,'product_price':price.text,'product_detail':detail.text,'product_url':img_url },ignore_index=True)
        
                
                

        
    except:
        Product=Product.append({'product_name' : "url not available"},ignore_index=True)
        continue
        
Product       
        
        

,product_name,product_price,product_detail,product_url
0,url not available,NaN,NaN,NaN
1,url not available,NaN,NaN,NaN
2,Old Spice Shaving Cream - 70 g (original) - Pa...,8,Health & Personal Care,https://m.media-amazon.com/images/I/41xHbjdSXJ...
3,url not available,NaN,NaN,NaN
4,url not available,NaN,NaN,NaN
...,...,...,...,...
106,url not available,NaN,NaN,NaN
107,url not available,NaN,NaN,NaN
108,url not available,NaN,NaN,NaN
109,url not available,NaN,NaN,NaN


In [10]:
Product.dropna()

,product_name,product_price,product_detail,product_url
2,Old Spice Shaving Cream - 70 g (original) - Pa...,8,Health & Personal Care,https://m.media-amazon.com/images/I/41xHbjdSXJ...
12,Old Spice Shaving Cream - 70 g (original) - Pa...,8,Health & Personal Care,https://m.media-amazon.com/images/I/41xHbjdSXJ...
26,Short Story: Violoncello und Klavier. (Cello-B...,€11.50,Books,https://images-na.ssl-images-amazon.com/images...
30,Concerto grosso g-Moll: Weihnachtskonzert. op....,€11.61,Books,https://images-na.ssl-images-amazon.com/images...
62,Concertino B-Dur: Violine und Orchester. Klavi...,€12.65,Books,https://images-na.ssl-images-amazon.com/images...
76,"Concerto in E flat ""Dumbarton Oaks"": für Kamme...",,Bücher,https://images-na.ssl-images-amazon.com/images...
77,Concert es kammerorchester parti musique d'ens...,,Livres,https://images-na.ssl-images-amazon.com/images...
80,Musik für Kinder: Im Fünftonraum. Vol. 1. Sing...,,Books,https://images-na.ssl-images-amazon.com/images...


In [11]:
df2 = Product.to_json(orient = 'columns')
print(df2)

{"product_name":{"0":"url not available","1":"url not available","2":"Old Spice Shaving Cream - 70 g (original) - Pack of 2","3":"url not available","4":"url not available","5":"url not available","6":"url not available","7":"url not available","8":"url not available","9":"url not available","10":"url not available","11":"url not available","12":"Old Spice Shaving Cream - 70 g (original) - Pack of 2","13":"url not available","14":"url not available","15":"url not available","16":"url not available","17":"url not available","18":"url not available","19":"url not available","20":"url not available","21":"url not available","22":"url not available","23":"url not available","24":"url not available","25":"url not available","26":"Short Story: Violoncello und Klavier. (Cello-Bibliothek)","27":"url not available","28":"url not available","29":"url not available","30":"Concerto grosso g-Moll: Weihnachtskonzert. op. 6\/8. 2 Violinen, Violoncello, Streichorchester und Cembalo. Partitur. (Concert